In [1]:
import glob
import pandas as pd
import numpy as np 
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import torch
from torch import nn
import random

# Personnal Import 
from utilities import DataSet, get_batch,get_mode_date2path
from load_data import load_subway_15_min

In [ ]:
# Validation Individuelles :                         [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M, , , , , , , , , ],[J,F,M, , , , , , , , , ]
# Subway 15 Min :   Novembre 2019 - Mai 2021         [ , , , , , , , , , ,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M, , , , , , , ]
# NetMob 15 Min :                                   [ , ,M,A,M, , , , , , , ],[ , , , , , , , , , , , ],[ , , , , , , , , , , , ]

# Data
- Validation individuelles, aggrégée 3min.
- Metro 15 min (entrée/sortie)

Idée : Identifier des coupure de métro longue :
    - entrée et sortie inhabituelles, montre que le métro est suspendu et laisse les portes ouvertes
    - Sortie plus faible (coupure > 15min)
Evaluer les qualités de prédiction sur ces moments là.

## Data Description
- Un 'VAL_ARRET_CODE' peut être l'arrêt de plusieurs mêmes bus, voir d'un même bus et d'un même arrêt de métro. Où d'un même bus et d'un même arrêt de tram. 
    - Je dois donc nommer différement les VAL_ARRET_CODE de chacun des modes. Une proposition est de mettre le mode (B,S,T) devant les id. Comme ça on pourra regrouper sans soucis.
- La moyenne des déplacement de la df_subway est de 5 trajet toute les 3 minutes, quelque soit la station et l'heure (d'ouverture) considéré. Max 88.

#### Questionnement 
- Ok on a aggrégé 3 min, mais est-ce qu'on peut recouper les sorties 3min avec les validation + Sortie de métro 15 min? 

## Load Validation Individuelles
Load 3 df : df_sub, df_tram, df_bus

In [5]:
folder_path = 'data/'
valid_ind_path = folder_path + 'Sub_Tram_11_2019_03_2020'
dates = ['11-2019','12-2019','1-2020','2-2020','3-2020']

subway_paths, tramway_paths, bus_paths = sorted(glob.glob(os.path.join(valid_ind_path, "*df_subway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_tramway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_bus*.csv")))
mode_month2path = get_mode_date2path([subway_paths,tramway_paths,bus_paths],['sub','tram','bus'])

for name in ['sub','tram','bus']:
    globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])

C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[

## Load Subway 15 min

In [10]:
txt_path = "Métro 15 minutes 2019 2020.txt"

df_metro_funi_2019_2020 = load_subway_15_min(folder_path+txt_path)
df_metro_funi_2019_2020.head()

,datetime,Station,Code ligne,in,out
0,2019-01-01 00:00:00,Ampère Victor Hugo,A,2,4.0
1,2019-01-01 00:15:00,Ampère Victor Hugo,A,3,2.0
2,2019-01-01 00:30:00,Ampère Victor Hugo,A,3,7.0
3,2019-01-01 00:45:00,Ampère Victor Hugo,A,1,9.0
4,2019-01-01 01:00:00,Ampère Victor Hugo,A,0,0.0


# 1er Etape : Prédiction Métro
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité. De la même manière que l'historique -7d sert de référence, mais ne témoigne pas d'un lien causal.
- Identifier des moments interessants : anomalies sur entrée/sortie métro. Voir les prédictions sur ces moments là particulier.


## Plot Coverage for each station, IN, OUT, and IN+OUT

#### Visualisation des flux IN et OUT entre 6h et 24H. En virant les outliers (0.95) type fête des lumières:

In [ ]:
# Import
from plotting import coverage_day_month

In [ ]:
# Init 
freq  = '15min'
columns = 'hour'
index = 'date'
quantile = 0.95

In [ ]:
# Plotting for each station 
for station in df_metro_funi_2019_2020.Station.unique():
    df_tmps = df_metro_funi_2019_2020[df_metro_funi_2019_2020.Station == station]
    df_tmps = df_tmps[df_tmps.datetime.dt.hour > 5]
    in99,out99 = df_tmps['in'].quantile(quantile),df_tmps['out'].quantile(quantile)
    df_tmps.loc[df_tmps['in']>in99,'in'] = in99
    df_tmps.loc[df_tmps['out']>out99,'out'] = out99
    coverage_day_month(df_tmps, freq = freq,index = index,columns = columns,save = station,folder_save = 'save/profile flux 15min filtred outliers 95/')

#### Visualisation des flux IN et OUT pour chacunes des stations, without filtering 

In [ ]:
for station in df_metro_funi_2019_2020.Station.unique():
    df_tmps = df_metro_funi_2019_2020[df_metro_funi_2019_2020.Station == station]
    coverage_day_month(df_tmps, freq = '60min',index = 'date',columns = 'hour',save = station,folder_save = 'save/profile flux')

# 1er Etape : Prédiction Métro
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité.
- Identifier des moment interessant : anomalie sur entrée/sortie métro. Voir les prédictions sur ces moments là particulier.

## Feature Vector
A définir pour chacune des stations

## Bla Bla : 

In [ ]:
#Bijecction entre un entier, et son label
def int2lab(int,n_adj=2,B=3):
    n = 1+(int-1)//B 
    b = int-(n-1)*B
    return(f'n{n}_b{b}')

def lab2int(lab,n_adj=2,B=3):
    nb = lab.split('_')
    n,b = int(nb[0][1:]),int(nb[1][1:])
    return((n-1)*B+b)

for k in range(1,15):
    print(int2lab(k),lab2int(int2lab(k)))

## Représentation des numéro des cellules du Tensor
Permet de faire des affichages graphique, et de s'assurer que les ".reshape" font ce que l'on souhaite

In [ ]:
def int2lab(integer, n_adj=2, B=3, C=4, N=5, L=6):
    N_adj_L = N * L
    N_adj_C_L = C * N_adj_L
    N_adj_B_C_L = B * N_adj_C_L

    l = 1 + ((integer - 1) % L)
    remaining = (integer - 1) // L
    n = 1 + (remaining % N)
    remaining //= N
    c = 1 + (remaining % C)
    remaining //= C
    b = 1 + (remaining % B)
    remaining //= B
    n_adj = 1 + remaining % n_adj

    return f'adj{n_adj}_b{b}_c{c}_n{n}_l{l}'

def lab2int(label, n_adj=2, B=3, C=4, N=5, L=6):
    split_label = label.split('_')
    n_adj = int(split_label[0][3:])
    b = int(split_label[1][1:])
    c = int(split_label[2][1:])
    n = int(split_label[3][1:])
    l = int(split_label[4][1:])

    N_adj_L = N * L
    N_adj_C_L = C * N_adj_L
    N_adj_B_C_L = B * N_adj_C_L

    integer = ((l - 1) + (n - 1) * L + (c - 1) * N * L + (b - 1) * C * N * L + (n_adj - 1) * B * C * N * L) + 1
    return integer


for k in range(1,400):
    print(int2lab(k),lab2int(int2lab(k)))

In [ ]:
label = int2lab(k)
label.split('_')

In [ ]:
permuted_H = output_H.permute(0, 1, 3, 4, 2)
print(permuted_H.shape)

new_c_in = permuted_H.shape[-1]
first_attn = permuted_H.reshape(K,-1,new_c_in)
print(first_attn.shape)  # [K, B*L*N, C_in']

attn_weight = nn.Linear(new_c_in,1)
softmax = nn.Softmax(-1)  #SoftMax on the last dimension 

#Coefficient d'attention 
attn = attn_weight(first_attn)    # "Embedding" du spatial channel 
attn = attn.permute(1,2,0)  # Permute pour avoir la nombre d'adjacency matrix en dernière dimension
attn = softmax(attn)  # Coefficient d'attention pour chaque Matrices d'adjacence et Embedding spatial assicié  (Ici 1 seule matrice d'adjacence)